In [1]:
import pandas as pd
import numpy as np

In [2]:
meta = pd.read_csv("data/movies_metadata.csv", low_memory=False)

In [3]:
print(meta.shape)
meta.head()

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
meta.iloc[0]

adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
id                                                                     862
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
poster_path                               /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     

In [5]:
meta.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [6]:
from ast import literal_eval

In [7]:
# convert complex data to list
meta['genres'] = meta['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
meta['production_companies']= meta['production_companies'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
meta['production_countries'] = meta['production_countries'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
meta['spoken_languages'] = meta['spoken_languages'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
meta['year'] = pd.to_datetime(meta['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [8]:
# delete null rows for important data
meta.drop(inplace=True, index=meta.index[meta['status'] != 'Released'])
meta.drop(inplace=True, index=meta.index[meta['release_date'].isnull()])
meta.drop(inplace=True, index=meta.index[meta['imdb_id'].isnull()])
meta.drop(inplace=True, index=meta.index[meta.imdb_id == '0'])
meta.drop(inplace=True, index=meta.index[meta['popularity'].isnull()])
meta.drop(inplace=True, index=meta.index[meta['title'].isnull()])
meta.drop(inplace=True, index=meta.index[meta['year'] == 'NaT'])
meta.drop(inplace=True, index=meta.index[meta['vote_count'].isnull()])
meta.drop(inplace=True, index=meta.index[meta['vote_average'].isnull()])

# delete useless columns
meta.drop(inplace=True, columns=["belongs_to_collection", "status", "budget", "original_title", "revenue", "video", "original_language", "release_date"])

# convert to int or float
meta['year'] = meta['year'].astype('int')
meta['vote_count'] = meta['vote_count'].astype('int')
meta['vote_average'] = meta['vote_average'].astype('float')
meta['popularity'] = meta['popularity'].astype('float')
meta['id'] = meta['id'].astype('int')

# delete duplicates
meta.drop(inplace=True, index=meta.index[meta.id.duplicated()])
meta.drop(inplace=True, index=meta.index[meta.imdb_id.duplicated()])

In [9]:
meta.sort_values('popularity', inplace=True, ascending=False)
meta.shape

(44895, 17)

In [10]:
meta = meta.head(500)

In [11]:
print(meta.shape)
print(meta.dtypes)
meta.head()

(500, 17)
adult                    object
genres                   object
homepage                 object
id                        int32
imdb_id                  object
overview                 object
popularity              float64
poster_path              object
production_companies     object
production_countries     object
runtime                 float64
spoken_languages         object
tagline                  object
title                    object
vote_average            float64
vote_count                int32
year                      int32
dtype: object


,adult,genres,homepage,id,imdb_id,overview,popularity,poster_path,production_companies,production_countries,runtime,spoken_languages,tagline,title,vote_average,vote_count,year
30700,False,"[Family, Animation, Adventure, Comedy]",http://www.minionsmovie.com/,211672,tt2293640,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,/q0R4crx2SehcEEQEkYObktdeFy.jpg,"[Universal Pictures, Illumination Entertainment]",[United States of America],91.0,[English],"Before Gru, they had a history of bad bosses",Minions,6.4,4729,2015
33356,False,"[Action, Adventure, Fantasy]",http://www.warnerbros.com/wonder-woman,297762,tt0451279,An Amazon princess comes to the world of Man t...,294.337037,/imekS7f1OuHyUP2LAiTEM0zBzUz.jpg,"[Dune Entertainment, Atlas Entertainment, Warn...",[United States of America],141.0,"[Deutsch, English]",Power. Grace. Wisdom. Wonder.,Wonder Woman,7.2,5025,2017
42222,False,"[Family, Fantasy, Romance]",http://movies.disney.com/beauty-and-the-beast-...,321612,tt2771200,A live-action adaptation of Disney's version o...,287.253654,/tWqifoYuwLETmmasnGHO7xBjEtt.jpg,"[Walt Disney Pictures, Mandeville Films]","[United Kingdom, United States of America]",129.0,[English],Be our guest.,Beauty and the Beast,6.8,5530,2017
43644,False,"[Action, Crime]",NaN,339403,tt3890160,After being coerced into working for a crime b...,228.032744,/dN9LbVNNZFITwfaRjl4tmwGWkRg.jpg,"[Big Talk Productions, TriStar Pictures, Media...","[United Kingdom, United States of America]",113.0,[English],All you need is one killer track.,Baby Driver,7.2,2083,2017
24455,False,"[Adventure, Family, Animation, Action, Comedy]",http://movies.disney.com/big-hero-6,177572,tt2245084,The special bond that develops between plus-si...,213.849907,/9gLu47Zw5ertuFTZaxXOvNfy78T.jpg,"[Walt Disney Pictures, Walt Disney Animation S...",[United States of America],102.0,[English],From the creators of Wreck-it Ralph and Frozen,Big Hero 6,7.8,6289,2014


In [12]:
meta[meta.title == "Titanic"]

,adult,genres,homepage,id,imdb_id,overview,popularity,poster_path,production_companies,production_countries,runtime,spoken_languages,tagline,title,vote_average,vote_count,year
1639,False,"[Drama, Romance, Thriller]",http://www.titanicmovie.com,597,tt0120338,"84 years later, a 101-year-old woman named Ros...",26.88907,/kHXEpyfl6zqn8a6YuozZUujufXf.jpg,"[Paramount Pictures, Twentieth Century Fox Fil...",[United States of America],194.0,"[English, Français, Deutsch, svenska, Italiano...",Nothing on Earth could come between them.,Titanic,7.5,7770,1997


In [13]:
meta[meta.title == "Toy Story"]

,adult,genres,homepage,id,imdb_id,overview,popularity,poster_path,production_companies,production_countries,runtime,spoken_languages,tagline,title,vote_average,vote_count,year
0,False,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],81.0,[English],NaN,Toy Story,7.7,5415,1995


In [14]:
meta[meta.title == "Inception"]

,adult,genres,homepage,id,imdb_id,overview,popularity,poster_path,production_companies,production_countries,runtime,spoken_languages,tagline,title,vote_average,vote_count,year
15480,False,"[Action, Thriller, Science Fiction, Mystery, A...",http://inceptionmovie.warnerbros.com/,27205,tt1375666,"Cobb, a skilled thief who commits corporate es...",29.108149,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,"[Legendary Pictures, Warner Bros., Syncopy]","[United Kingdom, United States of America]",148.0,[English],Your mind is the scene of the crime.,Inception,8.1,14075,2010


In [15]:
meta[meta.title == "The Dark Knight"]

,adult,genres,homepage,id,imdb_id,overview,popularity,poster_path,production_companies,production_countries,runtime,spoken_languages,tagline,title,vote_average,vote_count,year
12481,False,"[Drama, Action, Crime, Thriller]",http://thedarkknight.warnerbros.com/dvdsite/,155,tt0468569,Batman raises the stakes in his war on crime. ...,123.167259,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,"[DC Comics, Legendary Pictures, Warner Bros., ...","[United Kingdom, United States of America]",152.0,"[English, 普通话]",Why So Serious?,The Dark Knight,8.3,12269,2008


In [16]:
meta.shape

(500, 17)

In [17]:
s = meta.apply(lambda x: pd.Series(x['genres'], dtype='object'),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = meta.drop('genres', axis=1).join(s)
gen_md.head(8)

,adult,homepage,id,imdb_id,overview,popularity,poster_path,production_companies,production_countries,runtime,spoken_languages,tagline,title,vote_average,vote_count,year,genre
0,False,http://toystory.disney.com/toy-story,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],81.0,[English],NaN,Toy Story,7.7,5415,1995,Animation
0,False,http://toystory.disney.com/toy-story,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],81.0,[English],NaN,Toy Story,7.7,5415,1995,Comedy
0,False,http://toystory.disney.com/toy-story,862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],81.0,[English],NaN,Toy Story,7.7,5415,1995,Family
1,False,NaN,8844,tt0113497,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],104.0,"[English, Français]",Roll the dice and unleash the excitement!,Jumanji,6.9,2413,1995,Adventure
1,False,NaN,8844,tt0113497,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],104.0,"[English, Français]",Roll the dice and unleash the excitement!,Jumanji,6.9,2413,1995,Fantasy
1,False,NaN,8844,tt0113497,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],104.0,"[English, Français]",Roll the dice and unleash the excitement!,Jumanji,6.9,2413,1995,Family
5,False,NaN,949,tt0113277,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[Regency Enterprises, Forward Pass, Warner Bros.]",[United States of America],170.0,"[English, Español]",A Los Angeles Crime Saga,Heat,7.7,1886,1995,Action
5,False,NaN,949,tt0113277,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[Regency Enterprises, Forward Pass, Warner Bros.]",[United States of America],170.0,"[English, Español]",A Los Angeles Crime Saga,Heat,7.7,1886,1995,Crime


In [18]:
print(gen_md.shape)

(1439, 17)


In [19]:
def build_chart(genre, percentile=0.90):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [20]:
build_chart('Action').head(15)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.108149,7.373492
12481,The Dark Knight,2008,12269,8,123.167259,7.315751
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.173277
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.137875
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,7.104179
14551,Avatar,2009,12114,7,185.070892,6.687163
17818,The Avengers,2012,12000,7,89.887648,6.685318
26564,Deadpool,2016,11444,7,187.860492,6.675997
23753,Guardians of the Galaxy,2014,10014,7,53.291601,6.649277
26553,Mad Max: Fury Road,2015,9629,7,29.361780,6.641314


In [21]:
build_chart('Horror').head(15)

,title,year,vote_count,vote_average,popularity,wr
1213,The Shining,1980,3890,8,19.611589,7.110460
1171,Alien,1979,4564,7,23.377420,6.580581
41492,Split,2016,4461,7,28.920839,6.574726
1158,Aliens,1986,3282,7,21.761179,6.493853
42169,Get Out,2017,2978,7,36.894806,6.467755
21130,World War Z,2013,5683,6,26.114917,5.974396


In [22]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
351,Forrest Gump,1994,8147,8,48.307194,7.392781
1639,Titanic,1997,7770,7,26.889070,6.733918
40882,La La Land,2016,4745,7,19.681686,6.645618
23437,Maleficent,2014,4607,7,19.467404,6.640170
42222,Beauty and the Beast,2017,5530,6,287.253654,6.115159


## Content based

In [23]:
links_small = pd.read_csv('data/links.csv')
print(links_small.shape)
links_small.head()

(45843, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [24]:
links_small.drop(inplace=True, index=links_small.index[links_small['imdbId'].isnull()])
links_small.drop(inplace=True, index=links_small.index[links_small['tmdbId'].isnull()])
links_small['movie_id'] = links_small['movieId']
links_small.drop(inplace=True, columns=["movieId"])
print(links_small.shape)

(45624, 3)


In [25]:
smd = meta[meta['id'].isin(links_small.tmdbId)]
print(smd.shape)
smd.head()

(500, 17)


,adult,genres,homepage,id,imdb_id,overview,popularity,poster_path,production_companies,production_countries,runtime,spoken_languages,tagline,title,vote_average,vote_count,year
30700,False,"[Family, Animation, Adventure, Comedy]",http://www.minionsmovie.com/,211672,tt2293640,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,/q0R4crx2SehcEEQEkYObktdeFy.jpg,"[Universal Pictures, Illumination Entertainment]",[United States of America],91.0,[English],"Before Gru, they had a history of bad bosses",Minions,6.4,4729,2015
33356,False,"[Action, Adventure, Fantasy]",http://www.warnerbros.com/wonder-woman,297762,tt0451279,An Amazon princess comes to the world of Man t...,294.337037,/imekS7f1OuHyUP2LAiTEM0zBzUz.jpg,"[Dune Entertainment, Atlas Entertainment, Warn...",[United States of America],141.0,"[Deutsch, English]",Power. Grace. Wisdom. Wonder.,Wonder Woman,7.2,5025,2017
42222,False,"[Family, Fantasy, Romance]",http://movies.disney.com/beauty-and-the-beast-...,321612,tt2771200,A live-action adaptation of Disney's version o...,287.253654,/tWqifoYuwLETmmasnGHO7xBjEtt.jpg,"[Walt Disney Pictures, Mandeville Films]","[United Kingdom, United States of America]",129.0,[English],Be our guest.,Beauty and the Beast,6.8,5530,2017
43644,False,"[Action, Crime]",NaN,339403,tt3890160,After being coerced into working for a crime b...,228.032744,/dN9LbVNNZFITwfaRjl4tmwGWkRg.jpg,"[Big Talk Productions, TriStar Pictures, Media...","[United Kingdom, United States of America]",113.0,[English],All you need is one killer track.,Baby Driver,7.2,2083,2017
24455,False,"[Adventure, Family, Animation, Action, Comedy]",http://movies.disney.com/big-hero-6,177572,tt2245084,The special bond that develops between plus-si...,213.849907,/9gLu47Zw5ertuFTZaxXOvNfy78T.jpg,"[Walt Disney Pictures, Walt Disney Animation S...",[United States of America],102.0,[English],From the creators of Wreck-it Ralph and Frozen,Big Hero 6,7.8,6289,2014


In [26]:
print(smd.shape)

smd = pd.merge(smd, links_small.drop(columns=["imdbId"]).rename(columns={'tmdbId':'id'}), on='id', how='left')

print(smd.shape)

(500, 17)
(500, 18)


In [28]:
print(smd.index[smd.imdb_id.duplicated()])
smd.drop(inplace=True, index=smd.index[smd.imdb_id.duplicated()])
print(smd.index[smd.movie_id.duplicated()])
print(smd.index[smd.id.duplicated()])
print(smd.shape)
smd.head()

Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
(500, 18)


,adult,genres,homepage,id,imdb_id,overview,popularity,poster_path,production_companies,production_countries,runtime,spoken_languages,tagline,title,vote_average,vote_count,year,movie_id
0,False,"[Family, Animation, Adventure, Comedy]",http://www.minionsmovie.com/,211672,tt2293640,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,/q0R4crx2SehcEEQEkYObktdeFy.jpg,"[Universal Pictures, Illumination Entertainment]",[United States of America],91.0,[English],"Before Gru, they had a history of bad bosses",Minions,6.4,4729,2015,135887
1,False,"[Action, Adventure, Fantasy]",http://www.warnerbros.com/wonder-woman,297762,tt0451279,An Amazon princess comes to the world of Man t...,294.337037,/imekS7f1OuHyUP2LAiTEM0zBzUz.jpg,"[Dune Entertainment, Atlas Entertainment, Warn...",[United States of America],141.0,"[Deutsch, English]",Power. Grace. Wisdom. Wonder.,Wonder Woman,7.2,5025,2017,143355
2,False,"[Family, Fantasy, Romance]",http://movies.disney.com/beauty-and-the-beast-...,321612,tt2771200,A live-action adaptation of Disney's version o...,287.253654,/tWqifoYuwLETmmasnGHO7xBjEtt.jpg,"[Walt Disney Pictures, Mandeville Films]","[United Kingdom, United States of America]",129.0,[English],Be our guest.,Beauty and the Beast,6.8,5530,2017,168366
3,False,"[Action, Crime]",NaN,339403,tt3890160,After being coerced into working for a crime b...,228.032744,/dN9LbVNNZFITwfaRjl4tmwGWkRg.jpg,"[Big Talk Productions, TriStar Pictures, Media...","[United Kingdom, United States of America]",113.0,[English],All you need is one killer track.,Baby Driver,7.2,2083,2017,171763
4,False,"[Adventure, Family, Animation, Action, Comedy]",http://movies.disney.com/big-hero-6,177572,tt2245084,The special bond that develops between plus-si...,213.849907,/9gLu47Zw5ertuFTZaxXOvNfy78T.jpg,"[Walt Disney Pictures, Walt Disney Animation S...",[United States of America],102.0,[English],From the creators of Wreck-it Ralph and Frozen,Big Hero 6,7.8,6289,2014,115617


In [29]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2) ,min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [31]:
tfidf_matrix.shape

(500, 19535)

In [33]:
print(tfidf_matrix[0])

  (0, 1361)	0.17157267127234063
  (0, 8097)	0.17157267127234063
  (0, 7544)	0.17157267127234063
  (0, 19254)	0.17157267127234063
  (0, 13079)	0.17157267127234063
  (0, 7756)	0.17157267127234063
  (0, 7959)	0.17157267127234063
  (0, 8458)	0.17157267127234063
  (0, 8963)	0.17157267127234063
  (0, 722)	0.17157267127234063
  (0, 18507)	0.17157267127234063
  (0, 16654)	0.17157267127234063
  (0, 12460)	0.17157267127234063
  (0, 14924)	0.17157267127234063
  (0, 13998)	0.17157267127234063
  (0, 1931)	0.17157267127234063
  (0, 9447)	0.17157267127234063
  (0, 16524)	0.17157267127234063
  (0, 11343)	0.17157267127234063
  (0, 1996)	0.17157267127234063
  (0, 1358)	0.1244478311900746
  (0, 8096)	0.11857896075759324
  (0, 7543)	0.14747342416355577
  (0, 19213)	0.06593828740962895
  (0, 13069)	0.126736306454972
  (0, 7754)	0.16090858454268153
  (0, 7958)	0.17157267127234063
  (0, 8454)	0.13201412113671895
  (0, 8961)	0.14747342416355577
  (0, 721)	0.17157267127234063
  (0, 18505)	0.12924304748725232
 

In [334]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

array([1.        , 0.0059314 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.004493  , 0.        , 0.        ,
       0.        , 0.        , 0.00371294, 0.        , 0.        ,
       0.        , 0.        , 0.00368606, 0.        , 0.        ,
       0.        , 0.00400948, 0.        , 0.00360323, 0.00445249,
       0.        , 0.        , 0.        , 0.00449408, 0.        ,
       0.00396422, 0.        , 0.        , 0.        , 0.0108306 ,
       0.        , 0.00380933, 0.00574912, 0.0029868 , 0.        ,
       0.        , 0.        , 0.        , 0.00453858, 0.        ,
       0.        , 0.        , 0.00872777, 0.        , 0.        ,
       0.        , 0.00299736, 0.00768312, 0.        , 0.        ,
       0.00321188, 0.01841127, 0.        , 0.01210717, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00279448,
       0.03171727, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01629621, 0.     

In [335]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [336]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [337]:
movie='Star Wars'
print("Description of the Movie: ", movie)
print('---------------------------------------------------------------------')
print(smd[smd['title']==movie]['overview'])

Description of the Movie:  Star Wars
---------------------------------------------------------------------
45    Princess Leia is captured and held hostage by ...
Name: overview, dtype: object


In [338]:
get_recommendations('Star Wars').head(20)

294                 The Empire Strikes Back
96             Star Wars: The Force Awakens
224                                   Mulan
353     Prince of Persia: The Sands of Time
338    The Scorpion King: Rise of a Warrior
1                              Wonder Woman
23                          Alien: Covenant
244                             San Andreas
441                                  Alien³
331                              Die Hard 2
202                                Iron Man
287                                Hercules
362                            Donnie Darko
260                         Black Butterfly
277                              Cinderella
211                     Saving Private Ryan
355                            Passenger 57
4                                Big Hero 6
476                            Irreversible
188                               Gladiator
Name: title, dtype: object

In [339]:
movie='Toy Story'
print("Description of the Movie: ", movie)
print('---------------------------------------------------------------------')
print(smd[smd['title']==movie]['overview'])

Description of the Movie:  Toy Story
---------------------------------------------------------------------
206    Led by Woody, Andy's toys live happily in his ...
Name: overview, dtype: object


In [340]:
get_recommendations('Toy Story').head(20)

415                                 Toy Story 2
456                                 Toy Story 3
282                                Factory Girl
58     Harry Potter and the Philosopher's Stone
164                              Child's Play 3
231                              In a Heartbeat
33                     The Shawshank Redemption
260                             Black Butterfly
139                              Monsters, Inc.
258                              V for Vendetta
201                                Ghostbusters
101                                    Twilight
300                               Trainspotting
11               War for the Planet of the Apes
278                            Django Unchained
465                                   Peter Pan
494                   The Princess and the Frog
275                                Child's Play
77                                   Your Name.
359                                American Pie
Name: title, dtype: object

## Metadata Based

In [341]:
credits = pd.read_csv('data/credits.csv')
keywords = pd.read_csv('data/keywords.csv')

In [342]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [343]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [344]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
meta.shape

(500, 17)

In [345]:
keywords.drop(inplace=True, index=keywords.index[keywords.id.duplicated()])
credits.drop(inplace=True, index=credits.index[credits.id.duplicated()])

In [346]:
meta = meta.merge(credits, on='id')
meta = meta.merge(keywords, on='id')

In [348]:
print(keywords.shape)
print(credits.shape)
print(meta.shape)

(45432, 2)
(45432, 3)
(500, 20)


In [349]:
meta['cast'] = meta['cast'].apply(literal_eval)
meta['crew'] = meta['crew'].apply(literal_eval)
meta['keywords'] = meta['keywords'].apply(literal_eval)
meta['cast_size'] = meta['cast'].apply(lambda x: len(x))
meta['crew_size'] = meta['crew'].apply(lambda x: len(x))

In [350]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [351]:
meta['director'] = meta['crew'].apply(get_director)

In [352]:
meta['cast'] = meta['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
meta['cast'] = meta['cast'].apply(lambda x: x[:4] if len(x) >=4 else x)

In [353]:
meta['keywords'] = meta['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [354]:
meta['cast'] = meta['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [355]:
meta['director'] = meta['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
meta['director'] = meta['director'].apply(lambda x: [x,x,x])

In [359]:
s = meta.apply(lambda x: pd.Series(x['keywords'], dtype="object"),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [360]:
s = s.value_counts()
s[:10]

based on novel          54
3d                      46
dystopia                41
sequel                  41
duringcreditsstinger    40
superhero               38
aftercreditsstinger     37
based on comic          32
violence                30
murder                  28
Name: keyword, dtype: int64

In [361]:
s = s[s > 1]

In [362]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stemmer = SnowballStemmer('english')
stemmer.stem('sportingly')

'sport'

In [363]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [365]:
meta['keywords'] = meta['keywords'].apply(filter_keywords)
meta['keywords'] = meta['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
meta['keywords'] = meta['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [366]:
meta['soup'] = meta['keywords'] + meta['cast'] + meta['director'] + meta['genres']
meta['soup'] = meta['soup'].apply(lambda x: ' '.join(x))

In [368]:
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(meta['soup'])

In [369]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [370]:
smd = meta.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [371]:
get_recommendations('The Dark Knight').head(15)

243                 The Dark Knight Rises
122                         Batman Begins
458                          The Prestige
103                               Dunkirk
119                             Inception
90                           Interstellar
98     Batman v Superman: Dawn of Justice
448                        Batman & Robin
427                              Kick-Ass
250                 S.W.A.T.: Under Siege
490                   Law Abiding Citizen
37                The Fate of the Furious
316                                Batman
322      Undisputed II: Last Man Standing
445                 The Lego Batman Movie
Name: title, dtype: object

In [373]:
get_recommendations('Titanic').head(15)

193     Terminator 2: Judgment Day
210                         Aliens
304                 The Terminator
7                           Avatar
412               The Great Gatsby
88            Fifty Shades of Grey
216                         Enigma
375                     Annie Hall
38                    Forrest Gump
78     Dilwale Dulhania Le Jayenge
420                    Wild Orchid
244                    San Andreas
255                     Passengers
479                 The Handmaiden
77                      Your Name.
Name: title, dtype: object

In [376]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = meta.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.50)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [377]:
improved_recommendations('The Dark Knight')

<ipython-input-376-58af45385901>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_count'] = qualified['vote_count'].astype('int')
<ipython-input-376-58af45385901>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_average'] = qualified['vote_average'].astype('int')
<ipython-input-376-58af45385901>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,title,vote_count,vote_average,year,wr
119,Inception,14075,8,2010,7.740185
90,Interstellar,11187,8,2014,7.687284
458,The Prestige,4510,8,2006,7.409143
243,The Dark Knight Rises,9263,7,2012,6.882662
122,Batman Begins,7511,7,2005,6.863099
30,Logan,6310,7,2017,6.845432
1,Wonder Woman,5025,7,2017,6.820673
427,Kick-Ass,4747,7,2010,6.814236
215,Kill Bill: Vol. 2,4061,7,2004,6.796180
408,The Equalizer,2997,7,2014,6.760000


In [379]:
improved_recommendations('Titanic')

<ipython-input-376-58af45385901>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_count'] = qualified['vote_count'].astype('int')
<ipython-input-376-58af45385901>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_average'] = qualified['vote_average'].astype('int')
<ipython-input-376-58af45385901>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,title,vote_count,vote_average,year,wr
38,Forrest Gump,8147,8,1994,7.637683
7,Avatar,12114,7,2009,6.914085
345,The Departed,4455,7,2006,6.821353
193,Terminator 2: Judgment Day,4274,7,1991,6.816677
304,The Terminator,4208,7,1984,6.814910
412,The Great Gatsby,3885,7,2013,6.805749
409,Edward Scissorhands,3731,7,1990,6.801055
210,Aliens,3282,7,1986,6.785973
79,Me Before You,2674,7,2016,6.761490
142,The Twilight Saga: Breaking Dawn - Part 2,2641,6,2012,6.260000


## Collaborative filtering

In [380]:
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

reader = Reader()
ratings = pd.read_csv('data/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [381]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5)

{'test_rmse': array([0.89996827, 0.89374983, 0.90084808, 0.89611367, 0.89673251]),
 'test_mae': array([0.69362764, 0.68805272, 0.69393392, 0.68987382, 0.69130454]),
 'fit_time': (11.119025230407715,
  11.603112936019897,
  10.541975736618042,
  11.061759948730469,
  10.74437165260315),
 'test_time': (0.26842594146728516,
  0.2478024959564209,
  0.28029465675354004,
  0.6205828189849854,
  0.24933433532714844)}

In [382]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [384]:
user_ratings = pd.merge(ratings, meta, left_on='movieId', right_on='id', how='inner')
user_ratings_final = user_ratings[['userId', 'movieId', 'rating','title']]
user_ratings = user_ratings_final.sort_values(by='userId')
user_ratings.head()

,userId,movieId,rating,title
0,1,2105,4.0,American Pie
637,2,497,3.0,The Green Mile
697,2,539,3.0,Psycho
822,2,550,3.0,Fight Club
833,2,585,5.0,"Monsters, Inc."


In [385]:
user_ratings[user_ratings['userId'] == 7]

,userId,movieId,rating,title
237,7,272,3.0,Batman Begins
1388,7,207,3.0,Dead Poets Society
1416,7,671,4.0,Harry Potter and the Philosopher's Stone
958,7,141,4.0,Donnie Darko
1449,7,745,5.0,The Sixth Sense
698,7,539,3.0,Psycho


In [562]:
movie1 = meta['title'] == 'Batman Begins'
meta[movie1][['title','id']]

,title,id
122,Batman Begins,272


In [563]:
svd.predict(7, 272, 3)

Prediction(uid=7, iid=272, r_ui=3, est=3.7195763710383822, details={'was_impossible': False})

## Combination

In [531]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [551]:
id_map = pd.read_csv('data/links.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(meta[['title', 'id']], on='id').set_index('title')

In [553]:
print(id_map.shape)
id_map.head()

(500, 2)


,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Heat,6,949.0
Se7en,47,807.0
Braveheart,110,197.0


In [558]:
indices_map = id_map.set_index('id')

In [559]:
print(indices_map.shape)
indices_map.head()

(500, 1)


,movieId
id,
862.0,1
8844.0,2
949.0,6
807.0,47
197.0,110


In [576]:
svd.predict(7, indices_map.loc[272]['movieId'])

Prediction(uid=7, iid=33794, r_ui=None, est=3.801968023047378, details={'was_impossible': False})

In [583]:
def recommend_my_movie(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    movies = meta.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies.sort_values('est', ascending=False, inplace=True)
    return movies.head(10)

In [584]:
recommend_my_movie(7, "Inception")

,title,vote_count,vote_average,year,id,est
458,The Prestige,4510,8.0,2006,1124,4.073539
15,The Dark Knight,12269,8.3,2008,155,4.001368
194,2001: A Space Odyssey,3075,7.9,1968,62,3.979253
238,Minority Report,2663,7.1,2002,180,3.812448
122,Batman Begins,7511,7.5,2005,272,3.801968
90,Interstellar,11187,8.1,2014,157336,3.779642
182,Alien,4564,7.9,1979,348,3.679363
304,The Terminator,4208,7.4,1984,218,3.614378
243,The Dark Knight Rises,9263,7.6,2012,49026,3.569178
210,Aliens,3282,7.7,1986,679,3.518619


In [585]:
recommend_my_movie(25, "Inception")

,title,vote_count,vote_average,year,id,est
210,Aliens,3282,7.7,1986,679,3.968131
458,The Prestige,4510,8.0,2006,1124,3.841302
90,Interstellar,11187,8.1,2014,157336,3.822167
15,The Dark Knight,12269,8.3,2008,155,3.710925
304,The Terminator,4208,7.4,1984,218,3.672082
168,The Fifth Element,3962,7.3,1997,18,3.671810
182,Alien,4564,7.9,1979,348,3.550095
115,Mad Max: Fury Road,9629,7.3,2015,76341,3.534496
238,Minority Report,2663,7.1,2002,180,3.488439
348,Predator,2129,7.3,1987,106,3.483143


In [586]:
meta.title.loc[meta.popularity == meta.popularity.sort_index(kind="mergesort")].head(40)

0                                               Minions
1                                          Wonder Woman
2                                  Beauty and the Beast
3                                           Baby Driver
4                                            Big Hero 6
5                                              Deadpool
6                        Guardians of the Galaxy Vol. 2
7                                                Avatar
8                                             John Wick
9                                             Gone Girl
10                The Hunger Games: Mockingjay - Part 1
11                       War for the Planet of the Apes
12                           Captain America: Civil War
13                                         Pulp Fiction
14     Pirates of the Caribbean: Dead Men Tell No Tales
15                                      The Dark Knight
16                                         Blade Runner
17                                         The A